In [1]:
AntennaTableHeader = ['Licence Number', 'Antenna', 'Client','Device Type','Emission Center Frequency','Transmitter Power','Antenna Height (AGL)', 
                      'Antenna Polarisation','Antenna Azimuth', 'Antenna Tilt','Device Registration ID','Date Authorised']
AntennaTable = pd.DataFrame(columns = AntennaTableHeader)


for acmalink in ACMAdatalinks:
    url = 'https://web.acma.gov.au'+ acmalink
    # Create object page
    page = requests.get(url)
    soup2 = BeautifulSoup(page.text, 'lxml')
    
    Antennaheaders = [td.text for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('class')]
    AntennaValues = [td.text.strip() for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    
    if url[-2:]=='/1':
        Antennaheaders2 = [td.text for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('class')]
        AntennaValues2 = [td.text.strip() for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    
    #To_be_removed = ['Registration and Application Details - 1','Registration and Application Details - 2','Receiver Details', 'Registration and Application Details', 'Transmitter Details','Device Details','Assignment and Application Details']
    
    #for item in To_be_removed:
     #   if item in Antennaheaders:
    #        Antennaheaders.remove(item)
     #   if item in Antennaheaders2:
     #       Antennaheaders2.remove(item)
     



    AntennaDictionary = {Antennaheaders[i]: AntennaValues[i] for i in range(len(Antennaheaders))}
    AntennaTable_toAdd = pd.DataFrame([AntennaDictionary])
    AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd], ignore_index=True)
    if url[-2:]=='/1':
        AntennaTable_toAdd2 = pd.DataFrame([{Antennaheaders2[i]: AntennaValues2[i] for i in range(len(Antennaheaders))}])
        AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd2], ignore_index=True)

    #print(output.head())
    
    #AntennaTable = AntennaTable.append(AntennaDictionary, ignore_index=True)

In [2]:
##=============================================================================
## ACMAscrapper V1.0.2
##=============================================================================
##=============================================================================
##
## Scrapper for Register of Radiocommunications Licences of Australian Communications and Media Authority
# 
#@Author Haris Hassan
#@Email haris.hassan@radhaz.com.au 
#@linkedin https://www.linkedin.com/in/hassanharis/
#
##=============================================================================

# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [3]:
####Replace Site URL
site_url = 'https://web.acma.gov.au/pls/radcom/site_search.site_lookup?pSITE_ID=570900'

In [4]:
ACMApage = requests.get(site_url)
soup = BeautifulSoup(ACMApage.text, 'lxml')
table1 = soup.find('table',{"class": "tablelist responsive"})

In [5]:
ACMAtable = soup.find('table',{"class": "tablelist responsive"})
#links= [linkitem.get('href') for linkitem in table1.find_all('a', attrs={"title": "Assignment Details"})[1:]]
ACMAheaders = [td.text.strip() for td in ACMAtable.select('th')]
ACMAdata = pd.DataFrame(columns = ACMAheaders)

ACMAlinks = []
for j in ACMAtable.find_all('tr')[1:]:
    ACMAValues = [tv.text.strip() for tv in j.find_all('td')]
    ACMAdata.loc[len(ACMAdata)] = ACMAValues
    ACMAlinks.append([tl.get('href') for tl in j.find_all('a')][0])
ACMAdata.insert(1,'links', ACMAlinks )

In [6]:
ACMAdatalinks = list( dict.fromkeys(ACMAdata['links']) )

In [7]:
AntennaTableHeader = ['Licence Number', 'Antenna', 'Client','Device Type','Emission Center Frequency','Transmitter Power','Antenna Height (AGL)', 
                      'Antenna Polarisation','Antenna Azimuth', 'Antenna Tilt','Device Registration ID','Date Authorised']
AntennaTable = pd.DataFrame(columns = AntennaTableHeader)


for acmalink in ACMAdatalinks:
    url = 'https://web.acma.gov.au'+ acmalink
    # Create object page
    page = requests.get(url)
    soup2 = BeautifulSoup(page.text, 'lxml')
    
    Antennaheaders = [td.text for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('class')]
    AntennaValues = [td.text.strip() for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    
    if url[-2:]=='/1':
        Antennaheaders2 = [td.text for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('class')]
        AntennaValues2 = [td.text.strip() for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    
    #To_be_removed = ['Registration and Application Details - 1','Registration and Application Details - 2','Receiver Details', 'Registration and Application Details', 'Transmitter Details','Device Details','Assignment and Application Details']
    
    #for item in To_be_removed:
     #   if item in Antennaheaders:
    #        Antennaheaders.remove(item)
     #   if item in Antennaheaders2:
     #       Antennaheaders2.remove(item)
     



    AntennaDictionary = {Antennaheaders[i]: AntennaValues[i] for i in range(len(Antennaheaders))}
    AntennaTable_toAdd = pd.DataFrame([AntennaDictionary])
    AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd], ignore_index=True)
    if url[-2:]=='/1':
        AntennaTable_toAdd2 = pd.DataFrame([{Antennaheaders2[i]: AntennaValues2[i] for i in range(len(Antennaheaders))}])
        AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd2], ignore_index=True)

    #print(output.head())
    
    #AntennaTable = AntennaTable.append(AntennaDictionary, ignore_index=True)

In [8]:
display(AntennaTable[AntennaTableHeader])

In [9]:
AntennaTableHeader = ['Licence Number', 'Antenna', 'Client','Device Type','Emission Center Frequency','Transmitter Power','Antenna Height (AGL)', 
                      'Antenna Polarisation','Antenna Azimuth', 'Antenna Tilt','Device Registration ID','Date Authorised']
AntennaTable = pd.DataFrame(columns = AntennaTableHeader)


for acmalink in ACMAdatalinks:
    url = 'https://web.acma.gov.au'+ acmalink
    # Create object page
    page = requests.get(url)
    soup2 = BeautifulSoup(page.text, 'lxml')
    
    Antennaheaders = [td.text for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
    AntennaValues = [td.text.strip() for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    
    if url[-2:]=='/1':
        Antennaheaders2 = [td.text for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
        AntennaValues2 = [td.text.strip() for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    
    AntennaDictionary = {Antennaheaders[i]: AntennaValues[i] for i in range(len(Antennaheaders))}
    AntennaTable_toAdd = pd.DataFrame([AntennaDictionary])
    AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd], ignore_index=True)
    if url[-2:]=='/1':
        AntennaTable_toAdd2 = pd.DataFrame([{Antennaheaders2[i]: AntennaValues2[i] for i in range(len(Antennaheaders))}])
        AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd2], ignore_index=True)

    #print(output.head())
    
    #AntennaTable = AntennaTable.append(AntennaDictionary, ignore_index=True)

In [10]:
display(AntennaTable[AntennaTableHeader])

In [11]:
##=============================================================================
## ACMAscrapper V2.0.2
##=============================================================================
##=============================================================================
##
## Scrapper for Register of Radiocommunications Licences of Australian Communications and Media Authority
# 
#@Author Haris Hassan
#@Email haris.hassan@radhaz.com.au 
#@linkedin https://www.linkedin.com/in/hassanharis/
#
##=============================================================================

# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [12]:
####Replace Site URL
site_url = 'https://web.acma.gov.au/pls/radcom/site_search.site_lookup?pSITE_ID=402015'

In [13]:
ACMApage = requests.get(site_url)
soup = BeautifulSoup(ACMApage.text, 'lxml')
table1 = soup.find('table',{"class": "tablelist responsive"})

In [14]:
ACMAtable = soup.find('table',{"class": "tablelist responsive"})
#links= [linkitem.get('href') for linkitem in table1.find_all('a', attrs={"title": "Assignment Details"})[1:]]
ACMAheaders = [td.text.strip() for td in ACMAtable.select('th')]
ACMAdata = pd.DataFrame(columns = ACMAheaders)

ACMAlinks = []
for j in ACMAtable.find_all('tr')[1:]:
    ACMAValues = [tv.text.strip() for tv in j.find_all('td')]
    ACMAdata.loc[len(ACMAdata)] = ACMAValues
    ACMAlinks.append([tl.get('href') for tl in j.find_all('a')][0])
ACMAdata.insert(1,'links', ACMAlinks )

In [15]:
ACMAdatalinks = list( dict.fromkeys(ACMAdata['links']) )

In [16]:
AntennaTableHeader = ['Licence Number', 'Antenna', 'Client','Device Type','Emission Center Frequency','Transmitter Power','Antenna Height (AGL)', 
                      'Antenna Polarisation','Antenna Azimuth', 'Antenna Tilt','Device Registration ID','Date Authorised']
AntennaTable = pd.DataFrame(columns = AntennaTableHeader)


for acmalink in ACMAdatalinks:
    url = 'https://web.acma.gov.au'+ acmalink
    # Create object page
    page = requests.get(url)
    soup2 = BeautifulSoup(page.text, 'lxml')
    
    Antennaheaders = [td.text for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
    AntennaValues = [td.text.strip() for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    
    if url[-2:]=='/1':
        Antennaheaders2 = [td.text for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
        AntennaValues2 = [td.text.strip() for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    
    AntennaDictionary = {Antennaheaders[i]: AntennaValues[i] for i in range(len(Antennaheaders))}
    AntennaTable_toAdd = pd.DataFrame([AntennaDictionary])
    AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd], ignore_index=True)
    
    if url[-2:]=='/1':
        AntennaTable_toAdd2 = pd.DataFrame([{Antennaheaders2[i]: AntennaValues2[i] for i in range(len(Antennaheaders))}])
        AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd2], ignore_index=True)

In [17]:
display(AntennaTable[AntennaTableHeader])

In [18]:
AntennaTable.to_excel(r'C:\Users\Mewtwo\Desktop'+'\\'+ AntennaDictionary['Site'] + '.xlsx', columns = AntennaTableHeader, index=False)

In [19]:
#display(AntennaTable[AntennaTableHeader])

In [20]:
%notebook "C:\Users\Mewtwo\Documents\Python\ACMAscrapper.ipynb"